In [1]:
import pandas as pd

In [2]:
df_metadata = pd.read_csv('../attribute_extraction_metadata_template/Initial Attribute Definition for First Release - UPDATED SHEET .csv')

In [3]:
df_metadata = df_metadata[['category', 'attribute_field', 'category_attributevalue', 'description', 'max_multi_select', 'usage', 'entry mode']]

In [4]:
metadata_category = set(df_metadata.category)

In [5]:
category2config = {}
for i in metadata_category:
    df_tmp = df_metadata[df_metadata.category == i]
    category2config[i] = sorted(df_tmp.to_dict('records'), key=lambda x: x['attribute_field'])

In [6]:
df_samples = pd.read_csv('sdt887_product_attribution_data_one_listing_per_leaf_node__20230111.csv')

In [7]:
df_samples = df_samples[df_samples.category_path.apply(lambda x: x in metadata_category)]
df_samples = df_samples.sample(10, random_state=42)

In [8]:
from copy import deepcopy
import hashlib
from collections import OrderedDict
import json
def sortOD(od):
    res = OrderedDict()
    for k, v in sorted(od.items()):
        if isinstance(v, dict):
            res[k] = sortOD(v)
        else:
            res[k] = deepcopy(v)
    return res

In [10]:
res = []
for i in df_samples[['product_id', 'category_id', 'category_path', 'title', 'product_description', 'main_image_url']].to_dict('records'):
    i['text'] = \
f"""Title: {i['title']}
Description: {i['product_description']}"""
    i['attribute_config'] = category2config[i['category_path']]
    task_json = json.dumps(sortOD(i)).encode('utf-8')
    task_id = hashlib.md5(task_json).hexdigest()
    i['task_id'] = f"query_attribution_md_{task_id}"
    i['preloading_result'] = None
    res.append(i)

In [11]:
pd.DataFrame(res).to_json('example_offshore_input_listing_attr_extract_v2.json', lines=True, orient='records')